# Test Dickens
This notebook replicates the functionality of the `test_ollama.py` script.

In [ ]:
import os
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import gpt_4o_mini_complete, gpt_4o_complete, openai_embed
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
from lightrag.llm.siliconcloud import siliconcloud_embedding
from lightrag.utils import EmbeddingFunc
import asyncio
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import openai_complete_if_cache
from lightrag.llm.siliconcloud import siliconcloud_embedding
from lightrag.utils import EmbeddingFunc
import numpy as np


# Uncomment the below two lines if running in a jupyter notebook to handle the async nature of rag.insert()
import nest_asyncio
nest_asyncio.apply()

In [ ]:
WORKING_DIR = "./dickens"

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

# 在导入部分添加
from dotenv import load_dotenv

# 在代码的开头加载环境变量
load_dotenv()

In [11]:
async def llm_model_func(
    prompt, system_prompt=None, history_messages=[], keyword_extraction=False, **kwargs
) -> str:
    return await openai_complete_if_cache(
        "Qwen/Qwen2.5-7B-Instruct",
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key=os.getenv("SILICONFLOW_API_KEY"),
        base_url="https://api.siliconflow.cn/v1/",
        **kwargs,
    )


async def embedding_func(texts: list[str]) -> np.ndarray:
    return await siliconcloud_embedding(
        texts,
        model="netease-youdao/bce-embedding-base_v1",
        api_key=os.getenv("SILICONFLOW_API_KEY"),
        max_token_size=512,
    )



In [ ]:
# function test
async def test_funcs():
    result = await llm_model_func("How are you?")
    print("llm_model_func: ", result)

    result = await embedding_func(["How are you?"])
    print("embedding_func: ", result)


asyncio.run(test_funcs())


In [ ]:
rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=llm_model_func,
    embedding_func=EmbeddingFunc(
        embedding_dim=768, max_token_size=512, func=embedding_func
    ),
)


In [ ]:
# Choose the model here
model_choice = "ollama"  # Options: "ollama", "openai"

llm_model_func = ollama_model_complete
embedding_func = EmbeddingFunc(
    embedding_dim=1024,
    max_token_size=8192,
    func=lambda texts: ollama_embed(
        texts,
        embed_model="bge-m3:latest"
    )
)
rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=llm_model_func, 
    llm_model_name='qwen2.5:7b-instruct-q4_K_M',
    embedding_func=embedding_func,
    llm_model_max_token_size=32768,
    llm_model_kwargs={"host": "http://localhost:11434", "options": {"num_ctx": 32768}},

)

In [ ]:
with open("./book.txt") as f:
    rag.insert(f.read())

In [ ]:
# Perform naive search
print("naive.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="naive")))

In [ ]:
# Perform local search
print("local.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="local")))

In [ ]:
# Perform global search
print("global.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="global")))

In [ ]:
# Perform hybrid search
print("hybrid.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="hybrid")))

In [ ]:
# Perform mix search (Knowledge Graph + Vector Retrieval)
print("mix.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="mix")))